# **Don't forget to upvote if it helps 😊**

will be adding aditional infor to aid future 

## pytorch train code:
https://www.kaggle.com/a763337092/neural-network-starter-pytorch-version<br />
https://www.kaggle.com/a763337092/pytorch-resnet-starter-training

## tensorflow training code:
https://www.kaggle.com/code1110/jane-street-with-keras-nn-overfit

## reference notebook:
https://www.kaggle.com/a763337092/blending-tensorflow-and-pytorch

new way of determining finalisation 

# **adittional datasets used **
https://www.kaggle.com/jamespett/outputsfrommodelsontraining


reduced jane was an output file from the origional code in the  publicly avalible https://www.kaggle.com/jamespett/notebook857ba5a22d and itself is publicly avalible here - https://www.kaggle.com/jamespett/reduced-jane


additional models
https://www.kaggle.com/jamespett/models

In [ ]:
import os,gc
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import log_loss, roc_auc_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F


In [ ]:


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

DATA_PATH = '../input/jane-street-market-prediction/'

NFOLDS = 4

TRAIN = False
CACHE_PATH = '../input/mlp012003weights'

def save_pickle(dic, save_path):
    with open(save_path, 'wb') as f:
    # with gzip.open(save_path, 'wb') as f:
        pickle.dump(dic, f)

def load_pickle(load_path):
    with open(load_path, 'rb') as f:
    # with gzip.open(load_path, 'rb') as f:
        message_dict = pickle.load(f)
    return message_dict

feat_cols = [f'feature_{i}' for i in range(130)]

target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']

f_mean = np.load(f'{CACHE_PATH}/f_mean_online.npy')

##### Making features
all_feat_cols = [col for col in feat_cols]
all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])

##### Model&Data fnc
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(all_feat_cols))
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(all_feat_cols), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(all_feat_cols), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x
if True:
    device = torch.device("cuda:0")

    model_list = []
    tmp = np.zeros(len(feat_cols))
    for _fold in range(NFOLDS):
        torch.cuda.empty_cache()
        model = Model()
        model.to(device)
        model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
        model.load_state_dict(torch.load(model_weights))
        
        model.eval()
        model_list.append(model)
        del model

In [ ]:
THRESHOLD = 0.5
def precision(y_true, y_pred, threshold_shift=0.5-THRESHOLD):

    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))

    precision = tp / (tp + fp)
    return precision



embedings

In [ ]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import namedtuple
from sklearn.metrics import log_loss, roc_auc_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
N_FEAT_TAGS = 29    # No of tags in features.csv
DEVICE = device
N_FEATURES = 130
THREE_HIDDEN_LAYERS = [400, 400, 400]

class FFN (nn.Module):
    
    def __init__(self, inputCount=130, outputCount=5, hiddenLayerCounts=[150, 150, 150], 
                 drop_prob=0.2, nonlin=torch.nn.SiLU(), isOpAct=False):
        
        super(FFN, self).__init__()
        
        self.nonlin     = nonlin
        self.dropout    = nn.Dropout (drop_prob)
        self.batchnorm0 = nn.BatchNorm1d (inputCount)
        self.dense1     = nn.Linear (inputCount, hiddenLayerCounts[0])
        self.batchnorm1 = nn.BatchNorm1d (hiddenLayerCounts[0])
        self.dense2     = nn.Linear(hiddenLayerCounts[0], hiddenLayerCounts[1])
        self.batchnorm2 = nn.BatchNorm1d (hiddenLayerCounts[1])
        self.dense3     = nn.Linear(hiddenLayerCounts[1], hiddenLayerCounts[2])
        self.batchnorm3 = nn.BatchNorm1d (hiddenLayerCounts[2])        
        self.outDense   = None
        if outputCount > 0:
            self.outDense   = nn.Linear (hiddenLayerCounts[-1], outputCount)
        self.outActivtn = None
        if isOpAct:
            if outputCount == 1 or outputCount == 2:
                self.outActivtn = nn.Sigmoid ()
            elif outputCount > 0:
                self.outActivtn = nn.Softmax (dim=-1)
        return

    def forward (self, X):
        
        # X = self.dropout (self.batchnorm0 (X))
        X = self.batchnorm0 (X)
        X = self.dropout (self.nonlin (self.batchnorm1 (self.dense1 (X))))
        X = self.dropout (self.nonlin (self.batchnorm2 (self.dense2 (X))))
        X = self.dropout (self.nonlin (self.batchnorm3 (self.dense3 (X))))
        if self.outDense:
            X = self.outDense (X)
        if self.outActivtn:
            X = self.outActivtn (X)
        return X
    
    
class Emb_NN_Model (nn.Module):
    
    def __init__(self, three_hidden_layers=THREE_HIDDEN_LAYERS, embed_dim=(N_FEAT_TAGS), csv_file='../input/jane-street-market-prediction/features.csv'):
        
        super (Emb_NN_Model, self).__init__()
        global N_FEAT_TAGS
        N_FEAT_TAGS = 29
        
        # store the features to tags mapping as a datframe tdf, feature_i mapping is in tdf[i, :]
        dtype = {'tag_0' : 'int8'}
        for i in range (1, 29):
            k = 'tag_' + str (i)
            dtype[k] = 'int8'
        t_df = pd.read_csv (csv_file, usecols=range (1,N_FEAT_TAGS+1), dtype=dtype)
        t_df['tag_29'] = np.array ([1] + ([0] * (t_df.shape[0]-1)) ).astype ('int8')
        self.features_tag_matrix = torch.tensor (t_df.to_numpy ())
        N_FEAT_TAGS += 1
        
        # print ('self.features_tag_matrix =', self.features_tag_matrix)
        
        # embeddings for the tags. Each feature is taken a an embedding which is an avg. of its' tag embeddings
        self.embed_dim     = embed_dim
        self.tag_embedding = nn.Embedding (N_FEAT_TAGS+1, embed_dim) # create a special tag if not known tag for any feature
        self.tag_weights   = nn.Linear (N_FEAT_TAGS, 1)
        
        drop_prob          = 0.5
        self.ffn           = FFN (inputCount=(130+embed_dim), outputCount=0, hiddenLayerCounts=[(three_hidden_layers[0]+embed_dim), (three_hidden_layers[1]+embed_dim), (three_hidden_layers[2]+embed_dim)], drop_prob=drop_prob)
        self.outDense      = nn.Linear (three_hidden_layers[2]+embed_dim, 5)
        return
    
    def features2emb (self):
        """
        idx : int feature index 0 to N_FEATURES-1 (129)
        """
        
        all_tag_idxs = torch.LongTensor (np.arange (N_FEAT_TAGS)) #.to (DEVICE)              # (29,)
        tag_bools    = self.features_tag_matrix                                # (130, 29)
        # print ('tag_bools.shape =', tag_bools.size())
        f_emb        = self.tag_embedding (all_tag_idxs).repeat (130, 1, 1)    #;print ('1. f_emb =', f_emb) # (29, 7) * (130, 1, 1) = (130, 29, 7)
        # print ('f_emb.shape =', f_emb.size())
        f_emb        = f_emb * tag_bools[:, :, None]                           #;print ('2. f_emb =', f_emb) # (130, 29, 7) * (130, 29, 1) = (130, 29, 7)
        # print ('f_emb.shape =', f_emb.size())
        
        # Take avg. of all the present tag's embeddings to get the embedding for a feature
        s = torch.sum (tag_bools, dim=1)                                       # (130,)
        # print ('s =', s)              
        f_emb = torch.sum (f_emb, dim=-2) / s[:, None]                         # (130, 7)
        # print ('f_emb =', f_emb)        
        # print ('f_emb.shape =', f_emb.shape)
        
        # take a linear combination of the present tag's embeddings
        # f_emb = f_emb.permute (0, 2, 1)                                        # (130, 7, 29)
        # f_emb = self.tag_weights (f_emb)                      #;print ('3. f_emb =', f_emb)                 # (130, 7, 1)
        # f_emb = torch.squeeze (f_emb, dim=-1)                 #;print ('4. f_emb =', f_emb)                 # (130, 7)
        return f_emb
    
    def forward (self, cat_featrs, features):
        """
        when you call `model (x ,y, z, ...)` then this method is invoked
        """
        
        cat_featrs = None
        features   = features.view (-1, N_FEATURES)
        f_emb      = self.features2emb ()                                #;print ('5. f_emb =', f_emb); print ('6. features =', features) # (130, 7)
        # print ('features.shape =', features.shape, 'f_emb.shape =', f_emb.shape)
        features_2 = torch.matmul (features, f_emb)                      #;print ('7. features =', features) # (1, 130) * (130, 7) = (1, 7)
        # print ('features.shape =', features.shape)
        
        # Concatenate the two features (features + their embeddings)
        features   = torch.hstack ((features, features_2))        
        
        x          = self.ffn (features)                               #;print ('8. x.shape = ', x.shape, 'x =', x)   # (1, 7) -> (1, 7)
        # x        = self.layer_normal (x + features)                  #;print ('9. x.shape = ', x.shape, 'x =', x)   # (1, 7) -> (1, 2)
        out_logits = self.outDense (x)                                 #;print ('10. out_logits.shape = ', out_logits.shape, 'out_logits =', out_logits)        
        # return sigmoid probs
        # out_probs = F.sigmoid (out_logits)
        return out_logits

In [ ]:
embNN_model = Emb_NN_Model ()
try:
    embNN_model.load_state_dict (torch.load ("../input/janefastai-featinteraction-embeddingnn5-300x3/Jane_EmbNN5_auc_400_400_400.pth"))
except:
    embNN_model.load_state_dict (torch.load ("../input/janefastai-featinteraction-embeddingnn5-300x3/Jane_EmbNN5_auc_400_400_400.pth", map_location='cpu'))
    
embNN_model = embNN_model.eval ()

## tensorflow part

In [ ]:
import keras.backend as K
def profitable(y_true, y_pred):
    a=K.constant(0.5,dtype='float32')
    b=K.constant(1,dtype='float32')
    c=K.constant(0,dtype='float32')
    y_pred = K.greater(y_pred , a)
    y_true= K.greater(y_true , a)
    y_true_f = K.switch(K.equal(y_pred, y_true), c,K.switch(K.equal(y_pred, True), b, a))
    #y_true_f=
    return K.sum(y_true_f)


In [ ]:
!ls ../input/jane-street-with-keras-nn-overfit

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices


SEED = 1155

np.random.seed(SEED)

# fit
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

epochs = 200
batch_size = 4096
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

tf.keras.backend.clear_session()
tf.random.set_seed(SEED)
clf = create_mlp(
    len(feat_cols), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )
clf.load_weights('../input/jane-street-with-keras-nn-overfit/model.h5')

clf.call = tf.function(clf.call, experimental_relax_shapes=True)
#tf_models = [clf]
#del clf


a new way of determing how the model is working 

In [ ]:
'''
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
maxim = train['date'].max(axis=0) 
if (maxim>400):
    train = train.sample(frac=0.7).reset_index(drop = True) 
train = train[train['weight'] != 0]
features_mean = []
features = [c for c in train.columns if 'feature' in c]
for i in features:
    x = train[i].mean()  
    features_mean.append(x)
    train[i] = train[i].fillna(x)
train['action'] = ((train['resp'].values) *  np.sign(train['weight'])>0).astype(int)

train.to_csv("newtraining.csv")
'''
train = pd.read_csv('/kaggle/input/reduced-jane/newtraining.csv')
resp_cols = ['resp_1','resp_2', 'resp_3','resp', 'resp_4']
X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
newModTrainY=train.loc[:, train.columns.str.contains('action')]
#f_mean = np.mean(train[features[1:]].values,axis=0)
models=[]
addata =pd.DataFrame()
weight =train['weight']
del train
#aditional model 


epochs = 50
batch_size = 4098
hidden_units = [160,160,160]
dropout_rates = [0.2,0.2,0.2,0.2]
label_smoothing = 1e-2
learning_rate = 1e-3
torch.cuda.empty_cache() 




j=0
x_tt = X_train.values
cross_41_42_43 = x_tt[:, 41] + x_tt[:, 42] + x_tt[:, 43]
cross_1_2 = x_tt[:, 1] / (x_tt[:, 2] + 1e-5)
feature_inp = np.concatenate((
    x_tt,
    np.array(cross_41_42_43).reshape(x_tt.shape[0], 1),
    np.array(cross_1_2).reshape(x_tt.shape[0], 1),
    ), axis=1)
a=x_tt.shape[0]
del cross_41_42_43
del cross_1_2
'''
with torch.no_grad():
    torch_pred = np.zeros((a, len(target_cols)))
    for model in model_list:
        torch_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS
            
    addata['mod'+str(0)]=torch_pred[:,0]
    addata['mod'+str(1)]=torch_pred[:,1]
    addata['mod'+str(2)]=torch_pred[:,2]
    addata['mod'+str(3)]=torch_pred[:,3]
    addata['mod'+str(4)]=torch_pred[:,4]
    del torch_pred
    torch.cuda.empty_cache() 
'''
'''
ans =clf.predict(X_train)
addata['mod'+str(0)]=ans[:,0]
addata['mod'+str(1)]=ans[:,1]
addata['mod'+str(2)]=ans[:,2]
addata['mod'+str(3)]=ans[:,3]
addata['mod'+str(4)]=ans[:,4]
#j+=1
'''
'''
tf.config.optimizer.set_jit(True)
del j
del feature_inp
tfmodels=[]
for j in range(1,4):
    clf = tf.keras.models.load_model('../input/models/mod'+str(j))
    clf.call = tf.function(clf.call, experimental_relax_shapes=True)
    tfmodels.append(clf)
    del clf
    torch.cuda.empty_cache() 
addata['mod5'] ,addata['mod6'],addata['mod7']= [model(x_tt, training = False).numpy() for model in tfmodels]
torch.cuda.empty_cache() 
del x_tt
del X_train
del y_train
'''
#train0 = pd.read_csv('/kaggle/input/outputsfrommodelsontraining/predictions2.csv').iloc[:,1:]
train1 = pd.read_csv('/kaggle/input/outputsfrommodelsontraining/predictions1.csv').iloc[:,1:]
train2 = pd.read_csv('/kaggle/input/outputsfrommodelsontraining/predictions2.csv').iloc[:,1:]
train3=pd.concat([train1,train2],axis=1)

train3['weight']=weight
#train3=pd.concat([train3,train0],axis=1)

del train1
del train2

epochs = 15
batch_size = 4098
hidden_units = [5]
dropout_rates = [0.2,0.2]
label_smoothing = 1e-2
learning_rate = 1e-3
torch.cuda.empty_cache() 
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model
#metrics
#metrics=tf.keras.metrics.FalsePositives(thresholds=0.5),
finLayer = create_mlp(
    14, 1, hidden_units, dropout_rates, label_smoothing, learning_rate
    )
finLayer.fit(train3.to_numpy(), newModTrainY, epochs=epochs, batch_size=batch_size, verbose=2)

del newModTrainY
torch.cuda.empty_cache() 

tfmodels=[]
for j in range(1,4):
    clf1 = tf.keras.models.load_model('../input/models/mod'+str(j))
    clf1.call = tf.function(clf1.call, experimental_relax_shapes=True)
    tfmodels.append(clf1)
    del clf1
    torch.cuda.empty_cache() 



In [ ]:

finLayer.call = tf.function(finLayer.call, experimental_relax_shapes=True)

In [ ]:
import numpy as np
from numba import njit

@njit
def fillna_npwhere_njit(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [ ]:
from numba import jit
@jit(forceobj=True)
def prepro(test_df,feat_cols,f_mean):
    x_tt = test_df.loc[:, feat_cols].values
    if np.isnan(x_tt.sum()):
        x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean
    #fillna_npwhere_njit(x_tt, f_mean)
    cross_41_42_43 = x_tt[:, 41] + x_tt[:, 42] + x_tt[:, 43]
    cross_1_2 = x_tt[:, 1] / (x_tt[:, 2] + 1e-5)
    feature_inp = np.concatenate((
        x_tt,
        np.array(cross_41_42_43).reshape(x_tt.shape[0], 1),
        np.array(cross_1_2).reshape(x_tt.shape[0], 1),
    ), axis=1)

    return x_tt,feature_inp

In [ ]:
data=np.zeros((1,14))
K.set_learning_phase(False)
if True:
    import janestreet

    env = janestreet.make_env()
    env_iter = env.iter_test()

    for (test_df, pred_df) in tqdm(env_iter):
        if test_df['weight'].item() != 0:
            data[0,13]=test_df['weight']
            
            x_tt = test_df.loc[:, feat_cols].values
            if np.isnan(x_tt.sum()):
                x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean
            #fillna_npwhere_njit(x_tt, f_mean)
            cross_41_42_43 = x_tt[:, 41] + x_tt[:, 42] + x_tt[:, 43]
            cross_1_2 = x_tt[:, 1] / (x_tt[:, 2] + 1e-5)
            feature_inp = np.concatenate((
                x_tt,
                np.array(cross_41_42_43).reshape(x_tt.shape[0], 1),
                np.array(cross_1_2).reshape(x_tt.shape[0], 1),
            ), axis=1)
            
            #x_tt,feature_inp=prepro(test_df,feat_cols,f_mean)
            # torch_pred
            torch_pred = np.zeros((1, len(target_cols)))
            for model in model_list:
                torch_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS
            data[0,0:5] =torch_pred
            
            # tf_pred
            data[0,5:8] = [model(x_tt, training = False) for model in tfmodels]
            data[0,8:13]=clf(x_tt, training = False)
            # avg
            
            #x_tt1    = torch.tensor (x_tt[i,:]).float ().view (-1, 130)
            #data[0,14]= np.median (torch.sigmoid (embNN_model (None,x_tt1)).detach ().cpu ().numpy ().reshape ((-1, 5)), axis=1)
            pred = finLayer(data, training = False).numpy()
            pred_df.action = (pred[0] >= 0.5).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)

# Thank you